In [1]:
%reload_ext autoreload
%autoreload 2
import transformers
from transformers import AutoTokenizer
from datasets import DatasetDict, load_metric, Dataset
from code_.process_conll import process_file, extract_features
from code_.bert import Tokenizer, convert_to_dataset, compute_metrics, task, batch_size, model_checkpoint, get_labels_list_from_dataset
# from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
# from transformers import DataCollatorForTokenClassification


/Users/sezentuvay/Documents/adnlp/AdvancedNLP/Task_3/code_/bert.py:10: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
/Users/sezentuvay/opt/anaconda3/envs/myenvironment/lib/python3.12/site-packages/datasets/load.py:753: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [2]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

In [3]:
df_val = process_file('data/raw/en_ewt-up-dev.conllu')
df_train = process_file('data/raw/en_ewt-up-train.conllu')
df_test = process_file('data/raw/en_ewt-up-test.conllu')

process_file(): dataframe len: 4979
process_file(): dataframe len: 40498
process_file(): dataframe len: 4802


In [4]:
df_val = extract_features(df_val)
df_train = extract_features(df_train)
df_test = extract_features(df_test)

In [5]:
dataset = convert_to_dataset(df_train, df_val, df_test)
labels_list = get_labels_list_from_dataset(dataset)
print(labels_list)

['R-ARGM-MNR', 'ARGM-LVB', 'ARGM-MOD', 'C-ARGM-TMP', 'ARGM-LOC', 'C-ARGM-GOL', 'ARGM-TMP', 'R-ARG3', 'ARGM-MNR', 'C-ARG2', 'ARGM-PRR', 'C-ARG1', '_', 'ARGM-NEG', 'ARGM-EXT', 'ARGM-PRP', 'C-ARGM-LOC', 'C-ARGM-MNR', 'C-ARG0', 'ARGM-GOL', 'ARGM-ADV', 'R-ARG0', 'ARGM-ADJ', 'R-ARG1', 'C-ARG4', 'R-ARGM-ADV', 'C-ARGM-COM', 'C-ARGM-PRR', 'C-ARGM-EXT', 'R-ARG2', 'ARGM-REC', 'ARGA', 'C-ARGM-DIR', 'ARG2', 'ARG5', 'ARGM-PRD', 'R-ARGM-CAU', 'C-ARG1-DSP', 'ARG1-DSP', 'C-ARG3', 'ARG1', 'C-ARGM-ADV', 'R-ARG4', 'R-ARGM-LOC', 'V', 'R-ARGM-DIR', 'ARG3', 'ARG0', 'R-ARGM-COM', 'ARGM-DIS', 'ARGM-COM', 'ARGM-CXN', 'R-ARGM-TMP', 'C-ARGM-CXN', 'ARGM-DIR', 'ARGM-CAU', 'ARG4', 'R-ARGM-GOL', 'C-ARGM-PRP', 'C-V']


In [6]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [7]:

import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [11]:
example = dataset['train']
tokenized_inputs = tokenizer(example["sentences"], is_split_into_words=True)
input_ids = [id for sentence_ids in tokenized_inputs["input_ids"] for id in sentence_ids]
tokens = tokenizer.convert_ids_to_tokens(input_ids)


In [13]:
#print(tokens)

In [19]:
len(example["labels_list"]), len(tokenized_inputs["input_ids"])


(40498, 40498)

In [18]:
# word_ids = tokenized_inputs.word_ids()
# aligned_labels = [-100 if i is None else example['labels_list'][i] for i in word_ids]
# print(len(aligned_labels), len(tokenized_inputs["input_ids"]))

38 40498


In [22]:
label_all_tokens = True
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["sentences"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["sentences"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [24]:
#tokenize_and_align_labels(example)


In [28]:
#tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)


Map:   0%|          | 0/40498 [00:00<?, ? examples/s]

ArrowInvalid: Could not convert 'Al' with type str: tried to convert to int64